In [2]:
import numpy as np
from scipy.spatial.distance import cdist

# --- Original loop-based version ---
def update_support_loop(Y, X, wgts=None):
    n, d = X.shape
    N = np.sum(wgts) if wgts is not None else Y.shape[0]
    X1 = np.zeros_like(X)

    for i in range(n):
        xi = X[i, :]
        sum_term = np.zeros(d)
        for j in range(n):
            if i != j:
                diff = xi - X[j, :]
                sum_term += diff / np.linalg.norm(diff)
        X1[i, :] = (N / n) * sum_term

        q = 0.0
        for k, y in enumerate(Y):
            w = wgts[k] if wgts is not None else 1.0
            nm = np.linalg.norm(y - xi)
            X1[i, :] += w * y / nm
            q += w / nm
        X1[i, :] /= q

    return X1

# --- Vectorized version ---
def update_support_vec(Y, X, wgts=None):
    n, d = X.shape
    N = np.sum(wgts) if wgts is not None else Y.shape[0]

    diff = X[:, None, :] - X[None, :, :]  # shape (n, n, d)
    norm = np.linalg.norm(diff, axis=-1, keepdims=True) + 1e-10

    mask = ~np.eye(n, dtype=bool)
    masked_diff = (diff / norm) * mask[..., None]
    term1 = (N / n) * np.sum(masked_diff, axis=1)  # shape (n, d)

    dists = cdist(X, Y) + 1e-10
    weights = wgts if wgts is not None else np.ones(Y.shape[0])
    term2 = np.dot(dists**-1, (weights[:, None] * Y))  # shape (n, d)
    q = np.dot(dists**-1, weights)  # shape (n,)

    X1 = term1 + term2
    X1 /= q[:, None]
    return X1

# --- Test ---
np.random.seed(42)
Y = np.random.randn(200, 2)
X = np.random.randn(50, 2)
wgts = np.random.rand(Y.shape[0])

X1_loop = update_support_loop(Y, X, wgts)
X1_vec = update_support_vec(Y, X, wgts)

# Compare
print("Max abs diff:", np.max(np.abs(X1_loop - X1_vec)))
assert np.allclose(X1_loop, X1_vec, atol=1e-6), "Mismatch between loop and vectorized version!"
print("✅ The outputs match!")


Max abs diff: 6.291955845227903e-11
✅ The outputs match!
